In [ ]:
# azure-mgmt-sql(Manages Azure SQL resources), azure-identity(Handles Azure authentication)

In [ ]:
!pip install azure-mgmt-sql azure-identity azure-mgmt-resource pyodbc

In [ ]:
!pip install sqlalchemy pymssql

In [ ]:
# Steps to Log In Azure
# 1. Open Command Prompt (Windows) or Terminal (Mac/Linux)
# 2. Type and run: az login
# 3. A browser window will open asking you to sign in to your Azure account
# 4. After successful login, the terminal will display your Azure subscription details
# 5. Copy your subscription-id from command prompt

In [ ]:
# Azure Subscription ID
subscription_id = "<Azure Subscription ID Number>"

server = "nhl-azuresql-server.database.windows.net"
database_name = "NHL_AzureSQL_Database"
admin_user = "<user name>"
admin_password = "<password>"
driver = "ODBC Driver 18 for SQL Server"

In [24]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.sql import SqlManagementClient



# Authenticate using Azure CLI credentials
credential = DefaultAzureCredential()

# Initialize clients
resource_client = ResourceManagementClient(credential, subscription_id)
sql_client = SqlManagementClient(credential, subscription_id)

In [25]:
resource_group_name = "NHL_ResourceGroup"
location = "westus"

resource_client.resource_groups.create_or_update(resource_group_name, {"location": location})

print(f"✅ Resource Group '{resource_group_name}' created!")

✅ Resource Group 'NHL_ResourceGroup' created!


In [26]:
server_name = "nhl-azuresql-server"

server = sql_client.servers.begin_create_or_update(
    resource_group_name,
    server_name,
    {
        "location": location,
        "administrator_login": admin_user,
        "administrator_login_password": admin_password,
        "version": "12.0",
    }
).result()

print(f"✅ SQL Server '{server_name}' created!")

✅ SQL Server 'nhl-azuresql-server' created!


In [27]:
database_name = "NHL_AzureSQL-Database"

database = sql_client.databases.begin_create_or_update(
    resource_group_name,
    server_name,
    database_name,
    {
        "location": location,
        "sku": {"name": "Basic"},
    },
).result()

print(f"✅ Database '{database_name}' created!")

✅ Database 'NHL_AzureSQL-Database' created!


In [28]:
from azure.mgmt.sql import SqlManagementClient
from azure.mgmt.sql.models import FirewallRule
from azure.identity import DefaultAzureCredential

# from azure.identity import DefaultAzureCredential
# from azure.mgmt.sql import SqlManagementClient

# Authenticate using DefaultAzureCredential
credential = DefaultAzureCredential()
# subscription_id = 'your-subscription-id'

# Create a SQL Management client
client = SqlManagementClient(credential, subscription_id)

# Define parameters for the firewall rule
# server_name = 'your-sql-server-name'
# resource_group_name = 'your-resource-group-name'
firewall_rule_name = 'my-firewall-rule'
start_ip_address = '0.0.0.0'
end_ip_address = '255.255.255.255'

firewall_rule_params = FirewallRule(start_ip_address=start_ip_address, end_ip_address=end_ip_address)


# Create or update the firewall rule
firewall_rule = client.firewall_rules.create_or_update(
    resource_group_name=resource_group_name,
    server_name=server_name,
    firewall_rule_name=firewall_rule_name,
    parameters=firewall_rule_params
)

# Print meaningful output
print(f"Firewall rule '{firewall_rule.name}' created successfully.")
print(f"Start IP: {firewall_rule.start_ip_address}, End IP: {firewall_rule.end_ip_address}")

Firewall rule 'my-firewall-rule' created successfully.
Start IP: 0.0.0.0, End IP: 255.255.255.255


In [ ]:
# New Script

In [29]:
import pyodbc

# server = "nhl-azuresql-server.database.windows.net"

# Connection String
conn_str = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database_name};UID={admin_user};PWD={admin_password}"

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()


# Table creation queries (ensure correct order)
create_table_queries = [
    """
    CREATE TABLE team_info (
        team_id VARCHAR(3) PRIMARY KEY,
        franchise_id INT,
        short_name VARCHAR(15),
        team_name VARCHAR(15),
        abbreviation VARCHAR(15)
    )
    """,
    """
    CREATE TABLE player_info (
        player_id VARCHAR(10) PRIMARY KEY,
        first_name VARCHAR(20),
        last_name VARCHAR(20),
        nationality VARCHAR(3),
        birth_city VARCHAR(30),
        primary_position VARCHAR(3),
        birth_date DATE,
        birth_state_province VARCHAR(10),
        height_cm FLOAT,
        weight_kg FLOAT,
        shoots_catches VARCHAR(5)
    )
    """,
    """
    CREATE TABLE game (
        game_id VARCHAR(12) PRIMARY KEY,
        season INT,
        game_type VARCHAR(5),
        date_time_gmt DATETIME,
        away_team_id VARCHAR(3),
        home_team_id VARCHAR(3),
        away_goals INT,
        home_goals INT,
        home_rink_side_start VARCHAR(5),
        venue VARCHAR(50),
        venue_time_zone_id VARCHAR(20),
        venue_time_zone_offset INT,
        venue_time_zone_tz VARCHAR(3),
        hoa VARCHAR(4),
        settled_in VARCHAR(10)
    )
    """,
    """
    CREATE TABLE game_plays (
        play_id VARCHAR(15) PRIMARY KEY,
        game_id VARCHAR(12) REFERENCES game(game_id),
        team_id_for VARCHAR(3),
        team_id_against VARCHAR(3),
        event VARCHAR(25),
        secondary_type VARCHAR(25),
        x INT,
        y INT,
        period INT,
        period_type VARCHAR(20),
        period_time INT,
        period_time_remaining FLOAT,
        date_time DATETIME,
        goals_away INT,
        goals_home INT,
        description VARCHAR(255),
        st_x INT,
        st_y INT
    )
    """,
    """
    CREATE TABLE game_goals (
        game_goals_id INT IDENTITY(1,1) PRIMARY KEY,
        play_id VARCHAR(15) REFERENCES game_plays(play_id),
        strength VARCHAR(20),
        game_winning_goal VARCHAR(5),
        empty_net VARCHAR(5)
    )
    """,
    """
    CREATE TABLE game_teams_stats (
        game_teams_stats_id INT IDENTITY(1,1) PRIMARY KEY,
        game_id VARCHAR(12) REFERENCES game(game_id),
        team_id VARCHAR(3) REFERENCES team_info(team_id),
        hoa VARCHAR(5),
        won VARCHAR(3),
        settled_in VARCHAR(3),
        head_coach VARCHAR(50),
        goals INT,
        shots INT,
        hits FLOAT,
        penalty_minutes INT,
        power_play_opps INT,
        power_play_goals INT,
        face_off_win_percentage FLOAT,
        giveaways FLOAT, 
        takeaways FLOAT,
        blocked FLOAT,
        start_rink_side VARCHAR(5)
    )
    """,
    """
    CREATE TABLE game_plays_players (
        game_plays_players_id INT IDENTITY(1,1) PRIMARY KEY,
        play_id VARCHAR(15) REFERENCES game_plays(play_id),
        game_id VARCHAR(12) REFERENCES game(game_id),
        player_id VARCHAR(10) REFERENCES player_info(player_id),
        player_type VARCHAR(50)
    )
    """,
    """
    CREATE TABLE game_skater_stats (
        game_skater_stats_id INT IDENTITY(1,1) PRIMARY KEY,
        game_id VARCHAR(12) REFERENCES game(game_id),
        player_id VARCHAR(10) REFERENCES player_info(player_id),
        team_id VARCHAR(3) REFERENCES team_info(team_id),
        time_on_ice INT,
        assists INT,
        goals INT,
        shots INT,
        hits FLOAT,
        power_play_goals INT,
        power_play_assists INT,
        penalty_minutes INT,
        face_off_wins INT,
        face_off_taken INT,
        takeaways FLOAT,
        giveaways FLOAT,
        short_handed_goals INT,
        short_handed_assists INT,
        blocked FLOAT,
        plus_minus INT,
        even_time_on_ice INT,
        short_handed_time_on_ice INT,
        power_play_time_on_ice INT
    )
    """,
    """
    CREATE TABLE game_goalie_stats (
        game_goalie_stats_id INT IDENTITY(1,1) PRIMARY KEY,
        game_id VARCHAR(12) REFERENCES game(game_id),
        player_id VARCHAR(10) REFERENCES player_info(player_id),
        team_id VARCHAR(3) REFERENCES team_info(team_id),
        time_on_ice INT,
        assists INT,
        goals INT,
        penalty_minutes INT,
        shots INT,
        saves INT,
        power_play_saves INT,
        short_handed_saves INT,
        even_saves INT,
        short_handed_shots_against INT,
        even_shots_against INT,
        power_play_shots_against INT,
        decision VARCHAR(5),
        save_percentage FLOAT,
        power_play_save_percentage FLOAT,
        even_strength_save_percentage FLOAT
    )
    """
]

table_names = [
    "team_info", "player_info", "game", 
    "game_plays", "game_goals", "game_teams_stats",  
    "game_plays_players", "game_skater_stats", "game_goalie_stats"  
]

# Execute each table creation query
for table_name, query in zip(table_names, create_table_queries):
    try:
        cursor.execute(query)
        print(f"Table '{table_name}' created successfully.")
    except Exception as e:
        print(f"Error creating table '{table_name}': {e}")
        
# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()
print("All tables created successfully.")


Table 'team_info' created successfully.
Table 'player_info' created successfully.
Table 'game' created successfully.
Table 'game_plays' created successfully.
Table 'game_goals' created successfully.
Table 'game_teams_stats' created successfully.
Table 'game_plays_players' created successfully.
Table 'game_skater_stats' created successfully.
Table 'game_goalie_stats' created successfully.
All tables created successfully.


In [30]:
import os
import pandas as pd

In [31]:
folder_path = "CleanData"

In [32]:
table_names = [
    "team_info", "player_info", "game", 
    "game_plays", "game_goals", "game_teams_stats",  
    "game_plays_players", "game_skater_stats", "game_goalie_stats"  
]

In [33]:
# Create a dictionary to store DataFrames
dataframes = {}

In [34]:
for table in table_names:
    file_path = os.path.join(folder_path, f"{table}.csv")
    dataframes[table] = pd.read_csv(file_path, low_memory=False)
        
    print(f"Loaded {table}.csv into DataFrame '{table}'.")
    

Loaded team_info.csv into DataFrame 'team_info'.
Loaded player_info.csv into DataFrame 'player_info'.
Loaded game.csv into DataFrame 'game'.
Loaded game_plays.csv into DataFrame 'game_plays'.
Loaded game_goals.csv into DataFrame 'game_goals'.
Loaded game_teams_stats.csv into DataFrame 'game_teams_stats'.
Loaded game_plays_players.csv into DataFrame 'game_plays_players'.
Loaded game_skater_stats.csv into DataFrame 'game_skater_stats'.
Loaded game_goalie_stats.csv into DataFrame 'game_goalie_stats'.


In [35]:
# data type of dataframes
for df_name, df in dataframes.items():
    print(f"\nData types of '{df_name}':")
    print(df.dtypes)


Data types of 'team_info':
team_id          int64
franchise_id     int64
short_name      object
team_name       object
abbreviation    object
dtype: object

Data types of 'player_info':
player_id                 int64
first_name               object
last_name                object
nationality              object
birth_city               object
primary_position         object
birth_date               object
birth_state_province     object
height_cm               float64
weight_kg               float64
shoots_catches           object
dtype: object

Data types of 'game':
game_id                    int64
season                     int64
game_type                 object
date_time_gmt             object
away_team_id               int64
home_team_id               int64
away_goals                 int64
home_goals                 int64
home_rink_side_start      object
venue                     object
venue_time_zone_id        object
venue_time_zone_offset     int64
venue_time_zone_tz        ob

In [36]:
# print all dataframes
for df_name, df in dataframes.items():
    print(f"\nFirst 5 rows of '{df_name}':")
    print(df.head())


First 5 rows of 'team_info':
   team_id  franchise_id    short_name  team_name abbreviation
0        1            23    New Jersey     Devils          NJD
1        4            16  Philadelphia     Flyers          PHI
2       26            14   Los Angeles      Kings          LAK
3       14            31     Tampa Bay  Lightning          TBL
4        6             6        Boston     Bruins          BOS

First 5 rows of 'player_info':
   player_id first_name     last_name nationality      birth_city  \
0    8466148     Marian         Hossa         SVK   Stará Lubovna   
1    8465058     Michal      Rozsival         CZE          Vlasim   
2    8476906     Shayne  Gostisbehere         USA  Pembroke Pines   
3    8466285      Brian      Campbell         CAN       Strathroy   
4    8470607      Brent      Seabrook         CAN        Richmond   

  primary_position  birth_date birth_state_province  height_cm  weight_kg  \
0               RW  1979-01-12              unknown     185.42  93.8

In [ ]:
# Insert data into team_info Table

In [112]:
df_ti = dataframes["team_info"]

In [113]:
df_ti.head()

,team_id,franchise_id,short_name,team_name,abbreviation
0,1,23,New Jersey,Devils,NJD
1,4,16,Philadelphia,Flyers,PHI
2,26,14,Los Angeles,Kings,LAK
3,14,31,Tampa Bay,Lightning,TBL
4,6,6,Boston,Bruins,BOS


In [114]:
df_ti.shape

(33, 5)

In [54]:
import pandas as pd

# Connection String
conn_str = f"DRIVER={driver};SERVER={server};DATABASE={database_name};UID={admin_user};PWD={admin_password}"

try:
    # Connect to Azure SQL Database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

   #insert data into the database
    for index, row in df_ti.iterrows():
        cursor.execute("INSERT INTO team_info (team_id, franchise_id, short_name, team_name, abbreviation) VALUES (?, ?, ?, ?, ?)", 
                       row["team_id"], row["franchise_id"], row["short_name"], row["team_name"], row["abbreviation"])

    # Commit transaction
    conn.commit()
    print("Data inserted successfully!")

except Exception as e:
    print("Error: ", e)

finally:
    # Close connection
    cursor.close()
    conn.close()

Data inserted successfully!


In [ ]:
# Insert data into player_info Table

In [59]:
df_pi = dataframes["player_info"]

In [60]:
df_pi.head()

,player_id,first_name,last_name,nationality,birth_city,primary_position,birth_date,birth_state_province,height_cm,weight_kg,shoots_catches
0,8466148,Marian,Hossa,SVK,Stará Lubovna,RW,1979-01-12,unknown,185.42,93.893544,L
1,8465058,Michal,Rozsival,CZE,Vlasim,D,1978-09-03,unknown,185.42,95.254320,R
2,8476906,Shayne,Gostisbehere,USA,Pembroke Pines,D,1993-04-20,FL,180.34,81.646560,L
3,8466285,Brian,Campbell,CAN,Strathroy,D,1979-05-23,ON,177.80,87.089664,L
4,8470607,Brent,Seabrook,CAN,Richmond,D,1985-04-20,BC,190.50,99.790240,R


In [62]:
df_pi['birth_date'] = pd.to_datetime(df_pi['birth_date'])

In [74]:
df_pi['height_cm'] = df_pi['height_cm'].round(2)

In [75]:
df_pi['weight_kg'] = df_pi['weight_kg'].round(2)

In [76]:
df_pi.dtypes

player_id                        int64
first_name                      object
last_name                       object
nationality                     object
birth_city                      object
primary_position                object
birth_date              datetime64[ns]
birth_state_province            object
height_cm                      float64
weight_kg                      float64
shoots_catches                  object
dtype: object

In [77]:
df_pi.head()

,player_id,first_name,last_name,nationality,birth_city,primary_position,birth_date,birth_state_province,height_cm,weight_kg,shoots_catches
0,8466148,Marian,Hossa,SVK,Stará Lubovna,RW,1979-01-12,unknown,185.42,93.89,L
1,8465058,Michal,Rozsival,CZE,Vlasim,D,1978-09-03,unknown,185.42,95.25,R
2,8476906,Shayne,Gostisbehere,USA,Pembroke Pines,D,1993-04-20,FL,180.34,81.65,L
3,8466285,Brian,Campbell,CAN,Strathroy,D,1979-05-23,ON,177.80,87.09,L
4,8470607,Brent,Seabrook,CAN,Richmond,D,1985-04-20,BC,190.50,99.79,R


In [79]:
df_pi.isna().sum()

player_id                0
first_name               0
last_name                0
nationality              8
birth_city               5
primary_position         0
birth_date               0
birth_state_province     0
height_cm                3
weight_kg                3
shoots_catches          17
dtype: int64

In [80]:
has_empty_strings = (df_pi == "").any().any()

In [81]:
print(has_empty_strings)

False


In [85]:
df_pi = df_pi.where(pd.notnull(df_pi), None)

In [86]:
df_pi.head()

,player_id,first_name,last_name,nationality,birth_city,primary_position,birth_date,birth_state_province,height_cm,weight_kg,shoots_catches
0,8466148,Marian,Hossa,SVK,Stará Lubovna,RW,1979-01-12,unknown,185.42,93.89,L
1,8465058,Michal,Rozsival,CZE,Vlasim,D,1978-09-03,unknown,185.42,95.25,R
2,8476906,Shayne,Gostisbehere,USA,Pembroke Pines,D,1993-04-20,FL,180.34,81.65,L
3,8466285,Brian,Campbell,CAN,Strathroy,D,1979-05-23,ON,177.80,87.09,L
4,8470607,Brent,Seabrook,CAN,Richmond,D,1985-04-20,BC,190.50,99.79,R


In [87]:
null_rows = df_pi[df_pi['height_cm'].isnull()]

print(null_rows)

      player_id first_name last_name nationality     birth_city  \
1300    8479138      Scott    Foster        None           None   
1342    8480718        Ben    Wexler        None           None   
3541    8480356       Kyle    Keyser        None  Coral Springs   

     primary_position birth_date birth_state_province  height_cm  weight_kg  \
1300                G 1982-01-17              unknown        NaN        NaN   
1342                G 1994-12-23              unknown        NaN        NaN   
3541                G 1999-03-08                   FL        NaN        NaN   

     shoots_catches  
1300           None  
1342           None  
3541              L  


In [89]:
df_pi.shape

(3925, 11)

In [88]:
import pandas as pd
import pyodbc

# Connection String
conn_str = f"DRIVER={driver};SERVER={server};DATABASE={database_name};UID={admin_user};PWD={admin_password}"

# Function to insert data in chunks
def insert_data_in_player_info(df, chunk_size=1000):
    print("Processing Data insert...")
    try:
        # Connect to Azure SQL Database
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()

        # Insert data into the database in chunks
        for start in range(0, len(df), chunk_size):
            end = start + chunk_size
            chunk = df.iloc[start:end]
            for index, row in chunk.iterrows():
                cursor.execute("INSERT INTO player_info (player_id, first_name, last_name, nationality, "
                               "birth_city, primary_position, birth_date, birth_state_province, "
                               "height_cm, weight_kg, shoots_catches) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                               row["player_id"], row["first_name"], row["last_name"], row["nationality"], 
                               row["birth_city"], row["primary_position"], row["birth_date"], row["birth_state_province"], 
                               None if pd.isna(row["height_cm"]) else row["height_cm"], 
                               None if pd.isna(row["weight_kg"]) else row["weight_kg"], 
                               row["shoots_catches"])

        print(f"{end} rows inserted")

        # Commit transaction
        conn.commit()
        print("Data inserted successfully!")

    except Exception as e:
        print("Error: ", e)

    finally:
        # Close connection
        cursor.close()
        conn.close()

# Example usage
insert_data_in_player_info(df_pi, chunk_size=1000)

Processing Data insert...
4000 rows inserted
Data inserted successfully!


In [ ]:
# Insert data into game Table

In [90]:
df_g = dataframes["game"]

In [91]:
df_g.head()

,game_id,season,game_type,date_time_gmt,away_team_id,home_team_id,away_goals,home_goals,home_rink_side_start,venue,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz,hoa,settled_in
0,2016020045,20162017,R,2016-10-19 00:30:00+00:00,4,16,4,7,right,United Center,America/Chicago,-5,CDT,home,REG
1,2017020812,20172018,R,2018-02-07 00:00:00+00:00,24,7,4,3,left,KeyBank Center,America/New_York,-4,EDT,away,OT
2,2015020314,20152016,R,2015-11-24 01:00:00+00:00,21,52,4,1,right,MTS Centre,America/Winnipeg,-5,CDT,away,REG
3,2015020849,20152016,R,2016-02-17 00:00:00+00:00,52,12,1,2,right,PNC Arena,America/New_York,-4,EDT,home,REG
4,2017020586,20172018,R,2017-12-30 03:00:00+00:00,20,24,1,2,left,Honda Center,America/Los_Angeles,-7,PDT,home,REG


In [92]:
df_g.dtypes

game_id                    int64
season                     int64
game_type                 object
date_time_gmt             object
away_team_id               int64
home_team_id               int64
away_goals                 int64
home_goals                 int64
home_rink_side_start      object
venue                     object
venue_time_zone_id        object
venue_time_zone_offset     int64
venue_time_zone_tz        object
hoa                       object
settled_in                object
dtype: object

In [98]:
df_g['date_time_gmt'] = pd.to_datetime(df_g['date_time_gmt'])

In [99]:
df_g.dtypes

game_id                                 int64
season                                  int64
game_type                              object
date_time_gmt             datetime64[ns, UTC]
away_team_id                            int64
home_team_id                            int64
away_goals                              int64
home_goals                              int64
home_rink_side_start                   object
venue                                  object
venue_time_zone_id                     object
venue_time_zone_offset                  int64
venue_time_zone_tz                     object
hoa                                    object
settled_in                             object
dtype: object

In [100]:
df_g.isna().sum()

game_id                      0
season                       0
game_type                    0
date_time_gmt                0
away_team_id                 0
home_team_id                 0
away_goals                   0
home_goals                   0
home_rink_side_start      1099
venue                        0
venue_time_zone_id           0
venue_time_zone_offset       0
venue_time_zone_tz           0
hoa                          0
settled_in                   0
dtype: int64

In [101]:
has_empty_strings = (df_g == "").any().any()

In [102]:
print(has_empty_strings)

False


In [103]:
df_g = df_g.where(pd.notnull(df_g), None)

In [104]:
df_g.shape

(23735, 15)

In [110]:
import pyodbc
import pandas as pd

# Connection String
conn_str = f"DRIVER={driver};SERVER={server};DATABASE={database_name};UID={admin_user};PWD={admin_password}"

# Define chunk size
chunk_size = 1000

try:
    # Connect to Azure SQL Database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Insert data into the database in chunks
    for start in range(0, len(df_g), chunk_size):
        chunk = df_g.iloc[start:start + chunk_size]
        for index, row in chunk.iterrows():
            columns = ', '.join(row.index)
            placeholders = ', '.join(['?'] * len(row))
            sql = f"INSERT INTO game ({columns}) VALUES ({placeholders})"
            cursor.execute(sql, *row)

        # Commit transaction for each chunk
        conn.commit()
        print(f"Chunk {start // chunk_size + 1} inserted successfully!")

except Exception as e:
    print("Error: ", e)

finally:
    # Close connection
    cursor.close()
    conn.close()

Chunk 1 inserted successfully!
Chunk 2 inserted successfully!
Chunk 3 inserted successfully!
Chunk 4 inserted successfully!
Chunk 5 inserted successfully!
Chunk 6 inserted successfully!
Chunk 7 inserted successfully!
Chunk 8 inserted successfully!
Chunk 9 inserted successfully!
Chunk 10 inserted successfully!
Chunk 11 inserted successfully!
Chunk 12 inserted successfully!
Chunk 13 inserted successfully!
Chunk 14 inserted successfully!
Chunk 15 inserted successfully!
Chunk 16 inserted successfully!
Chunk 17 inserted successfully!
Chunk 18 inserted successfully!
Chunk 19 inserted successfully!
Chunk 20 inserted successfully!
Chunk 21 inserted successfully!
Chunk 22 inserted successfully!
Chunk 23 inserted successfully!
Chunk 24 inserted successfully!


In [ ]:
# Insert data into game_plays Table

In [133]:
df_gp = dataframes["game_plays"]

In [134]:
df_gp.head()

,play_id,game_id,team_id_for,team_id_against,event,secondary_type,x,y,period,period_type,period_time,period_time_remaining,date_time,goals_away,goals_home,description,st_x,st_y
0,2016020045_1,2016020045,NaN,NaN,Game Scheduled,NaN,NaN,NaN,1,REGULAR,0,1200.0,2016-10-18 23:40:58,0,0,Game Scheduled,NaN,NaN
1,2016020045_2,2016020045,NaN,NaN,Period Ready,NaN,NaN,NaN,1,REGULAR,0,1200.0,2016-10-19 01:35:28,0,0,Period Ready,NaN,NaN
2,2016020045_3,2016020045,NaN,NaN,Period Start,NaN,NaN,NaN,1,REGULAR,0,1200.0,2016-10-19 01:40:50,0,0,Period Start,NaN,NaN
3,2016020045_4,2016020045,16.0,4.0,Faceoff,NaN,0.0,0.0,1,REGULAR,0,1200.0,2016-10-19 01:40:50,0,0,Jonathan Toews faceoff won against Claude Giroux,0.0,0.0
4,2016020045_5,2016020045,16.0,4.0,Shot,Wrist Shot,-71.0,9.0,1,REGULAR,54,1146.0,2016-10-19 01:41:44,0,0,Artem Anisimov Wrist Shot saved by Michal Neuv...,71.0,-9.0


In [135]:
df_gp.shape

(4217063, 18)

In [136]:
df_gp.dtypes

play_id                   object
game_id                    int64
team_id_for              float64
team_id_against          float64
event                     object
secondary_type            object
x                        float64
y                        float64
period                     int64
period_type               object
period_time                int64
period_time_remaining    float64
date_time                 object
goals_away                 int64
goals_home                 int64
description               object
st_x                     float64
st_y                     float64
dtype: object

In [137]:
df_gp.isna().sum()

play_id                        0
game_id                        0
team_id_for               775247
team_id_against           775247
event                          0
secondary_type           3216072
x                         975973
y                         975945
period                         0
period_type                    0
period_time                    0
period_time_remaining     193019
date_time                      0
goals_away                     0
goals_home                     0
description                    0
st_x                      975973
st_y                      975973
dtype: int64

In [138]:
has_empty_strings = (df_gp == "").any().any()

In [139]:
print(has_empty_strings)

False


In [140]:
df_gp = df_gp.where(pd.notnull(df_gp), None)

In [142]:
df_gp['date_time'] = pd.to_datetime(df_gp['date_time'])

In [143]:
df_gp.dtypes

play_id                          object
game_id                           int64
team_id_for                     float64
team_id_against                 float64
event                            object
secondary_type                   object
x                               float64
y                               float64
period                            int64
period_type                      object
period_time                       int64
period_time_remaining           float64
date_time                datetime64[ns]
goals_away                        int64
goals_home                        int64
description                      object
st_x                            float64
st_y                            float64
dtype: object

In [144]:
df_gp.head()

,play_id,game_id,team_id_for,team_id_against,event,secondary_type,x,y,period,period_type,period_time,period_time_remaining,date_time,goals_away,goals_home,description,st_x,st_y
0,2016020045_1,2016020045,NaN,NaN,Game Scheduled,None,NaN,NaN,1,REGULAR,0,1200.0,2016-10-18 23:40:58,0,0,Game Scheduled,NaN,NaN
1,2016020045_2,2016020045,NaN,NaN,Period Ready,None,NaN,NaN,1,REGULAR,0,1200.0,2016-10-19 01:35:28,0,0,Period Ready,NaN,NaN
2,2016020045_3,2016020045,NaN,NaN,Period Start,None,NaN,NaN,1,REGULAR,0,1200.0,2016-10-19 01:40:50,0,0,Period Start,NaN,NaN
3,2016020045_4,2016020045,16.0,4.0,Faceoff,None,0.0,0.0,1,REGULAR,0,1200.0,2016-10-19 01:40:50,0,0,Jonathan Toews faceoff won against Claude Giroux,0.0,0.0
4,2016020045_5,2016020045,16.0,4.0,Shot,Wrist Shot,-71.0,9.0,1,REGULAR,54,1146.0,2016-10-19 01:41:44,0,0,Artem Anisimov Wrist Shot saved by Michal Neuv...,71.0,-9.0


In [151]:
max_length = df_gp['secondary_type'].str.len().max()

In [152]:
print(max_length)

38.0


In [154]:
# Alter table

import pyodbc

# Connection String
conn_str = f"DRIVER={driver};SERVER={server};DATABASE={database_name};UID={admin_user};PWD={admin_password}"

try:
    # Connect to Azure SQL Database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # SQL command to alter the table
    alter_table_sql = """
    ALTER TABLE game_plays 
    ALTER COLUMN secondary_type VARCHAR(50);  -- Adjust size as needed
    """

    # Execute the SQL command
    cursor.execute(alter_table_sql)
    conn.commit()
    print("Table altered successfully!")

except Exception as e:
    print("Error: ", e)

finally:
    # Close connection
    cursor.close()
    conn.close()

Table altered successfully!


In [ ]:
# Azure Logout
# In Command Prompt Type and run: az logout or az logout --username <your-email>